Analysis results for trajectory efficiency and reaction time
- Info: pseudonym, opponent pseudonym, date of the experiment
- Trajectory efficiency
- RT: from time choice first seen to time trajectory first starts being aligned with choice

Access other folders

In [43]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
main_copy_dir = os.path.join(parent_dir, "main_copy")
octagon_analysis = os.path.join(parent_dir, "octagon_analysis")
sys.path.append(main_copy_dir)
sys.path.append(octagon_analysis)

print("Added paths to sys.path:", main_copy_dir, octagon_analysis)

Added paths to sys.path: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy /Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis


Imports with autoreload

In [44]:
%load_ext autoreload
%autoreload 2

import analysis.trajectory_efficiency as trajectory_efficiency
import pandas as pd
import numpy as np
import parse_data.prepare_data as prepare_data
import parse_data.identify_filepaths as identify_filepaths
import prepare_questionnaire_data
import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define data folder and personal data file path

In [45]:
data_folder = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files'
personal_data_csv = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/personal_data.csv'

Generate the personal data dataframe

In [ ]:
personal_data_df = prepare_questionnaire_data.process_personal_data()

Prepare the questionnaire data

In [47]:
questionnaire_data_df = prepare_questionnaire_data.preprocess_data()

AQ_scores.csv
GAD_scores.csv
BIS_scores.csv


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny

Store synonyms in list

In [48]:
pseudos_list = [personal_data_df['pseudonym'].values]

Get social and solo filenames (solos ordered by pseudonym, neither ordered by date)

In [49]:
social_files, ordered_solos = identify_filepaths.get_filenames(data_folder=data_folder)

Separate first and second solos into lists

In [50]:
first_solos = ordered_solos[::2]
second_solos = ordered_solos[1::2]

Create list of tuples for pseudonyms in socials

In [ ]:
import re
pseudo_tuples_list = []
experiment_dates_socials = []
for file in social_files:
     # match the session number and the pseudonym string
    match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?_.*?)_Social\.json', file)
    if match:
        session, date, time, pseudonyms = match.groups()
        pseudo_tuples = pseudonyms.split('_')[0::]
        pseudo_tuples_list.append(pseudo_tuples)
        experiment_dates_socials.append(date)

In [145]:
social_dates = {}
opponent_dict = {}

for i, pseudonym in enumerate(pseudo_tuples_list):
    player0 = pseudonym[0]
    player1 = pseudonym[1]

    if player0 not in social_dates:
        social_dates[player0] = []
        opponent_dict[player0] = []
    if player1 not in social_dates:
        social_dates[player1] = []
        opponent_dict[player1] = []

    social_dates[player0] = experiment_dates_socials[i]
    social_dates[player1] = experiment_dates_socials[i]

    opponent_dict[player0] = player1
    opponent_dict[player1] = player0


Prepare data and get trial lists

In [ ]:
df_first_solos, first_solos_lists = prepare_data.prepare_data(data_folder, first_solos, combine=False)
df_second_solos, second_solos_lists = prepare_data.prepare_data(data_folder, second_solos, combine=False)
df_socials, social_lists = prepare_data.prepare_data(data_folder, social_files, combine=False)

*Trajectory efficiency*

Compute trajectory efficiency rations for all sessions in social lists

In [ ]:
trajectory_efficiency_ratios_list = []
for social_list in social_lists:   
        # Get the trajectory efficiency ratios for the session
    trajectory_efficiency_ratios = trajectory_efficiency.ratio_direct_to_absolute_distances_session(social_list)
    trajectory_efficiency_ratios_list.append(trajectory_efficiency_ratios)

In [ ]:
trajectory_efficiency_ratios_list

Store trajectory efficiency ratios for losses and wins for each pseudonym

In [14]:
loss_trajectory_efficiency_ratio = {}
win_trajectory_efficiency_ratio = {}

for i, tuple in enumerate(pseudo_tuples_list):
    player0 = tuple[0]
    player1 = tuple[1]
    
    if player0 not in win_trajectory_efficiency_ratio:
        win_trajectory_efficiency_ratio[player0]= []
        loss_trajectory_efficiency_ratio[player0] = []

    if player1 not in win_trajectory_efficiency_ratio:
        win_trajectory_efficiency_ratio[player1] = []
        loss_trajectory_efficiency_ratio[player1] = []
    
    win_trajectory_efficiency_ratio[player0] = trajectory_efficiency_ratios_list[i][0]
    loss_trajectory_efficiency_ratio[player0] = trajectory_efficiency_ratios_list[i][1]
    win_trajectory_efficiency_ratio[player1] = trajectory_efficiency_ratios_list[i][2]
    loss_trajectory_efficiency_ratio[player1] = trajectory_efficiency_ratios_list[i][3]

Compute trajectory efficiency for solos

In [ ]:
first_solo_trajectory_efficiency_ratios = trajectory_efficiency.ratio_direct_to_absolute_distances_solo_sessions(first_solos_lists)
second_solo_trajectory_efficiency_ratios = trajectory_efficiency.ratio_direct_to_absolute_distances_solo_sessions(second_solos_lists)

Get ordered pseudonym lists for first and second solos separately

In [116]:
first_solo_pseudo_list = []
second_solo_pseudo_list = []
first_solo_dates = []
second_solo_dates = []
solo_dates = {}


for first_solo, second_solo in zip(first_solos, second_solos):
   # match the session number and the pseudonym string
    first_match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?)_FirstSolo\.json', first_solo)
    second_match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?)_SecondSolo\.json', second_solo)

    if first_match:
        session, date, time, pseudonyms = first_match.groups()
        first_pseudo = pseudonyms.split('_')[0::]
        first_solo_pseudo_list.append(first_pseudo)
        first_solo_dates.append(date)

        
    if second_match:
        session, date, time, pseudonyms = second_match.groups()
        second_pseudo = pseudonyms.split('_')[0::]
        second_solo_pseudo_list.append(second_pseudo)
        second_solo_dates.append(date)

for i, pseudonym in enumerate(first_solo_pseudo_list):
    player = pseudonym[0]
    if player not in solo_dates:
        solo_dates[player] = []
    solo_dates[player] = first_solo_dates[i]




Store efficiency ratios for losses and wins for each pseudonym

In [18]:
first_solo_trajectory_efficiency_ratio = {}
second_solo_trajectory_efficiency_ratio = {}

for i, pseudonym in enumerate(first_solo_pseudo_list):
    player = pseudonym
    
    if player not in first_solo_trajectory_efficiency_ratio:
        first_solo_trajectory_efficiency_ratio[player]= []

    if player not in second_solo_trajectory_efficiency_ratio:
        second_solo_trajectory_efficiency_ratio[player]= []

    first_solo_trajectory_efficiency_ratio[player] = first_solo_trajectory_efficiency_ratios[i]
    second_solo_trajectory_efficiency_ratio[player] = second_solo_trajectory_efficiency_ratios[i]



Store overall trajectory efficiency results

In [153]:
trajectory_analysis_results = {
    pseudonym: {
        'socials_trajectory_efficiency': {
            'loss_trajectory_efficiency_ratio': loss_trajectory_efficiency_ratio[pseudonym],
            'win_trajectory_efficiency_ratio': win_trajectory_efficiency_ratio[pseudonym],
            'experiment_date': social_dates[pseudonym],
            'opponent': opponent_dict[pseudonym]
        },
        'solos_trajectory_efficiency': {
            'first_solo_efficiency_ratio': first_solo_trajectory_efficiency_ratio.get(pseudonym,np.nan),
            'second_solo_efficiency_ratio': second_solo_trajectory_efficiency_ratio.get(pseudonym,np.nan),
            'experiment_date': solo_dates.get(pseudonym,np.nan)
        }
    }
    for pseudonym in win_trajectory_efficiency_ratio
}

*Reaction time*

1. Filter trials by retrievable choice
2. Identify choice in each trial
3. Determine when (application time) choice wall is first seen 
4. Determine when player trajectory first starts being aligned with the choice wall

Additional imports

In [20]:
import trajectory_analysis.trajectory_direction as trajectory_direction
import trajectory_analysis.trajectory_vectors as trajectory_vectors
import data_extraction.extract_trial as extract_trial
import trajectory_analysis.trajectory_headangle as trajectory_headangle
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
import data_extraction.trial_list_filters as trial_list_filters
import data_extraction.get_indices as get_indices
import analysis.loser_inferred_choice as loser_inferred_choice
import questionnaire_analysis.questionnaire_data.reaction_time as reaction_time
import globals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Issues: losers choice may not be high wall nor low wall
- filtering by losing and winning trials will help with this
- can have np.nan when choice is not in walls

Socials

In [ ]:
rt_list = []
mean_rt_list = []

for index in range(len(social_lists)):
    trial_list = social_lists[index]

    reaction_times_p0, clean_reaction_times_p0 = reaction_time.reaction_time_session(trial_list, 0, 110)
    reaction_times_p1, clean_reaction_times_p1 = reaction_time.reaction_time_session(trial_list, 1, 110)

    mean_reaction_time_p0 = np.mean(clean_reaction_times_p0)
    mean_reaction_time_p1 = np.mean(clean_reaction_times_p1)
    
    mean_rt_list.append((mean_reaction_time_p0, mean_reaction_time_p1))

    rt_list.append((reaction_times_p0, reaction_times_p1))

Solos

In [ ]:
rt_list_first_solo = []
mean_rt_list_first_solo = []

for index in range(len(first_solos_lists)):
    trial_list = first_solos_lists[index]

    reaction_times_p0, clean_reaction_times_p0 = reaction_time.reaction_time_session(trial_list, 0, 110, solo=True)

    mean_reaction_time_p0 = np.mean(clean_reaction_times_p0)
    
    mean_rt_list_first_solo.append((mean_reaction_time_p0))

    rt_list_first_solo.append((reaction_times_p0))

In [ ]:
rt_list_second_solo = []
mean_rt_list_second_solo = []

for index in range(len(second_solos_lists)):
    trial_list = second_solos_lists[index]

    reaction_times_p0, clean_reaction_times_p0 = reaction_time.reaction_time_session(trial_list, 0, 110, solo=True)

    mean_reaction_time_p0 = np.mean(clean_reaction_times_p0)
    
    mean_rt_list_second_solo.append((mean_reaction_time_p0))

    rt_list_second_solo.append((reaction_times_p0))

In [ ]:
rt_list
# shape:
# tuple of player 0 and player 1 lists of reaction times for each trial list
# i.e., list of tuples of lists

In [ ]:
mean_rt_list
# shape:
# list of tuples of player 0 and player 1 mean reaction time for each trial list

In [26]:
rts_player0 = []
rts_player1 = []
# so if I only wanted player 1 reaction times I would do
for i in range(len(social_lists)):
    rts_player0.append(rt_list[i][0])
    rts_player1.append(rt_list[i][1])

In [27]:
m_rts_player0 = []
m_rts_player1 = []
# so if I only wanted player 1 reaction times I would do
for i in range(len(social_lists)):
    m_rts_player0.append(mean_rt_list[i][0])
    m_rts_player1.append(mean_rt_list[i][1])

In [28]:
trial_list_example, _ = trial_list_filters.filter_trials_retrievable_choice(social_lists[0], 0, inferred_choice=True, original_indices=None, debug=False)

winners_session = get_indices.get_trigger_activators(trial_list_example)
losers_session = (winners_session - 1) * -1
reaction_times_p0_wins = np.where(winners_session == 0, rts_player0[0], np.nan)
reaction_times_p0_losses = np.where(losers_session == 0, rts_player0[0], np.nan)


In [29]:
# assign pseudonyms to rt results

# presumably this should be looped over trial lists
# or extract right values in tuples
reaction_times_wins = {}
mean_reaction_time_wins = {}
reaction_times_losses = {}
mean_reaction_time_losses = {}

for i, tuple in enumerate(pseudo_tuples_list):
    player0 = tuple[0]
    player1 = tuple[1]

    if player0 not in reaction_times_wins:
        reaction_times_wins[player0] = []
        mean_reaction_time_wins[player0] = []
        reaction_times_losses[player0] = []
        mean_reaction_time_losses[player0] = []

    if player1 not in reaction_times_wins:
        reaction_times_wins[player1] = []
        mean_reaction_time_wins[player1] = []
        reaction_times_losses[player1] = []
        mean_reaction_time_losses[player1] = []

    trial_list = social_lists[i]

    trial_list_player0, _ = trial_list_filters.filter_trials_retrievable_choice(trial_list, 0, inferred_choice=True, original_indices=None, debug=False)
    trial_list_player1, _ = trial_list_filters.filter_trials_retrievable_choice(trial_list, 1, inferred_choice=True, original_indices=None, debug=False)

    winners_session_player0 = get_indices.get_trigger_activators(trial_list_player0)
    losers_session_player0 = (winners_session_player0 - 1) * -1

    winners_session_player1 = get_indices.get_trigger_activators(trial_list_player1)
    losers_session_player1 = (winners_session_player1 - 1) * -1

    reaction_times_p0_wins = np.where(winners_session_player0 == 0, rts_player0[i], np.nan)
    reaction_times_p0_losses = np.where(losers_session_player0 == 0, rts_player0[i], np.nan)

    reaction_times_p1_wins = np.where(winners_session_player1 == 1, rts_player1[i], np.nan)
    reaction_times_p1_losses = np.where(losers_session_player1 == 1, rts_player1[i], np.nan)

    reaction_times_wins[player0] = reaction_times_p0_wins
    reaction_times_wins[player1] = reaction_times_p1_wins

    reaction_times_losses[player0] = reaction_times_p0_losses
    reaction_times_losses[player1] = reaction_times_p1_losses

    mean_reaction_time_wins[player0] = np.mean([i for i in reaction_times_p0_wins if not np.isnan(i)])
    mean_reaction_time_wins[player1] = np.mean([i for i in reaction_times_p1_wins if not np.isnan(i)])

    mean_reaction_time_losses[player0] = np.mean([i for i in reaction_times_p0_losses if not np.isnan(i)])
    mean_reaction_time_losses[player1] = np.mean([i for i in reaction_times_p1_losses if not np.isnan(i)])

In [30]:
len(rt_list_first_solo)

70

In [31]:
# assign pseudonyms to rt results

rt_first_solo = {}
m_rt_first_solo = {}
rt_second_solo = {}
m_rt_second_solo = {}

for i, pseudonym in enumerate(first_solo_pseudo_list):
    player = pseudonym

    if player not in rt_first_solo:
        rt_first_solo[player] = []
        m_rt_first_solo[player] = []

    rt_first_solo[player] = rt_list_first_solo[i]
    m_rt_first_solo[player] = mean_rt_list_first_solo[i]

for i, pseudonym in enumerate(second_solo_pseudo_list):
    player = pseudonym

    if player not in rt_second_solo:
        rt_second_solo[player] = []
        m_rt_second_solo[player] = []

    rt_second_solo[player] = rt_list_second_solo[i]
    m_rt_second_solo[player] = mean_rt_list_second_solo[i]

In [32]:
rt_analysis_results_socials = {
    pseudonym: {
        'socials_reaction_times': {
            'reaction_times_wins': reaction_times_wins[pseudonym],
            'reaction_times_losses': reaction_times_losses[pseudonym],
            'mean_reaction_time_wins': mean_reaction_time_wins[pseudonym],
            'mean_reaction_time_losses': mean_reaction_time_losses[pseudonym]
        }
    }
    for pseudonym in reaction_times_wins
}

In [33]:
rt_analysis_results_solos = {
    pseudonym: {
        'solos_reaction_times': {
            'first_solo_reaction_time': rt_first_solo[pseudonym],
            'second_solo_reaction_time': rt_second_solo[pseudonym],
            'first_solo_mean_reaction_time': m_rt_first_solo[pseudonym],
            'second_solo_mean_reaction_time': m_rt_second_solo[pseudonym]
        }
    }
    for pseudonym in rt_first_solo
}

In [34]:
test_max=np.max([x for x in rt_analysis_results_socials['AL01']['socials_reaction_times']['reaction_times_wins'] if not np.isnan(x)])

In [35]:
test_min=np.min([x for x in rt_analysis_results_socials['AL01']['socials_reaction_times']['reaction_times_wins'] if not np.isnan(x)])

*Summary metric*

1. Normalise RT to be between 0 and 1 (to match trajectory efficiency ratio)
normalised_rt = 1 - ((rt - min_rt) / (max_rt - min_rt))
2. Take the product of the normalised RT and trajectory efficiency for each pseudonym as a summary metric

Notes: 
- Removed trials where choice is not retrievable from trajectory efficiency results
- traj efficiency is separated into winning and losing trials so separated RTs accordingly


Part 1)

In [36]:

norm_mean_rt_dict_wins = {}
norm_mean_rt_dict_losses = {}
for pseudonym in rt_analysis_results_socials:

    if pseudonym not in norm_mean_rt_dict_wins:
        norm_mean_rt_dict_wins[pseudonym] = []
        norm_mean_rt_dict_losses[pseudonym] = []

    reaction_time_win=rt_analysis_results_socials[pseudonym]['socials_reaction_times']['reaction_times_wins']
    min_rt_win = np.min([x for x in reaction_time_win if not np.isnan(x)])
    max_rt_win = np.max([x for x in reaction_time_win if not np.isnan(x)])

    norm_rts_wins = []
    for rt in reaction_time_win:
        norm_rt_win = 1-((rt-min_rt_win)/(max_rt_win-min_rt_win))
        norm_rts_wins.append(norm_rt_win)
    
    reaction_time_loss=rt_analysis_results_socials[pseudonym]['socials_reaction_times']['reaction_times_losses']
    min_rt_loss = np.min([x for x in reaction_time_loss if not np.isnan(x)])
    max_rt_loss = np.max([x for x in reaction_time_loss if not np.isnan(x)])

    norm_rts_losses = []
    for rt in reaction_time_loss:
        norm_rt_loss = 1-((rt-min_rt_loss)/(max_rt_loss-min_rt_loss))
        norm_rts_losses.append(norm_rt_loss)
        
    
    norm_mean_rt_wins = np.mean([i for i in norm_rts_wins if not np.isnan(i)])
    norm_mean_rt_dict_wins[pseudonym] = norm_mean_rt_wins

    norm_mean_rt_losses = np.mean([i for i in norm_rts_losses if not np.isnan(i)])
    norm_mean_rt_dict_losses[pseudonym] = norm_mean_rt_losses




In [37]:

norm_mean_rt_dict_first_solos = {}
norm_mean_rt_dict_second_solos = {}

for pseudonym in rt_analysis_results_solos:

    if pseudonym not in norm_mean_rt_dict_first_solos:
        norm_mean_rt_dict_first_solos[pseudonym] = []
        norm_mean_rt_dict_second_solos[pseudonym] = [] 

    reaction_time_first_solo = rt_analysis_results_solos[pseudonym]['solos_reaction_times']['first_solo_reaction_time']
    min_rt_first_solo = np.min([x for x in reaction_time_first_solo if not np.isnan(x)])
    max_rt_first_solo = np.max([x for x in reaction_time_first_solo if not np.isnan(x)])

    reaction_time_second_solo = rt_analysis_results_solos[pseudonym]['solos_reaction_times']['second_solo_reaction_time']
    min_rt_second_solo = np.min([x for x in reaction_time_second_solo if not np.isnan(x)])
    max_rt_second_solo = np.max([x for x in reaction_time_second_solo if not np.isnan(x)])

    norm_rts_first_solos = []
    for rt in reaction_time_first_solo:
        norm_rt = 1-((rt-min_rt_first_solo)/(max_rt_first_solo-min_rt_first_solo))
        norm_rts_first_solos.append(norm_rt)

    norm_rts_second_solos = []
    for rt in reaction_time_second_solo:
        norm_rt = 1-((rt-min_rt_second_solo)/(max_rt_second_solo-min_rt_second_solo))
        norm_rts_second_solos.append(norm_rt)
    
    norm_mean_rt_first = np.mean([i for i in norm_rts_first_solos if not np.isnan(i)])
    norm_mean_rt_dict_first_solos[pseudonym] = norm_mean_rt_first

    norm_mean_rt_second = np.mean([i for i in norm_rts_second_solos if not np.isnan(i)])
    norm_mean_rt_dict_second_solos[pseudonym] = norm_mean_rt_second

Part 2)

In [38]:
summary_scores_social_win = {}
summary_scores_social_loss = {}

for pseudonym in rt_analysis_results_socials:
    if pseudonym not in summary_scores_social_win:
        summary_scores_social_win[pseudonym] = []
        summary_scores_social_loss[pseudonym] = []
    
    summary_scores_social_win[pseudonym] = norm_mean_rt_dict_wins[pseudonym] * trajectory_analysis_results[pseudonym]['socials_trajectory_efficiency']['win_trajectory_efficiency_ratio']
    summary_scores_social_loss[pseudonym] = norm_mean_rt_dict_losses[pseudonym] * trajectory_analysis_results[pseudonym]['socials_trajectory_efficiency']['loss_trajectory_efficiency_ratio']

summary_scores_first_solo = {}
summary_scores_second_solo = {}

for pseudonym in first_solo_pseudo_list:
    if pseudonym not in summary_scores_first_solo:
        summary_scores_first_solo[pseudonym] = []
    if pseudonym in norm_mean_rt_dict_first_solos and pseudonym in trajectory_analysis_results:
        summary_scores_first_solo[pseudonym] = norm_mean_rt_dict_first_solos[pseudonym] * trajectory_analysis_results[pseudonym]['solos_trajectory_efficiency']['first_solo_efficiency_ratio']
    else: print(f"pseudonym {pseudonym} is missing from one of the dictionarites")

for pseudonym in second_solo_pseudo_list:
    if pseudonym not in summary_scores_second_solo:
        summary_scores_second_solo[pseudonym] = []
    if pseudonym in norm_mean_rt_dict_second_solos and pseudonym in trajectory_analysis_results:
        summary_scores_second_solo[pseudonym] = norm_mean_rt_dict_second_solos[pseudonym] * trajectory_analysis_results[pseudonym]['solos_trajectory_efficiency']['second_solo_efficiency_ratio']
    else: print(f"pseudonym {pseudonym} is missing from one of the dictionarites")

pseudonym JL14 is missing from one of the dictionarites
pseudonym JL14 is missing from one of the dictionarites


In [155]:
summary_scores_social = {
    pseudonym: {
        'rt_by_traj_efficiency_wins': summary_scores_social_win[pseudonym],
        'rt_by_traj_efficiency_losses': summary_scores_social_loss[pseudonym],
        'experiment_date': social_dates[pseudonym],
        'opponent': opponent_dict[pseudonym]
    }
    for pseudonym in summary_scores_social_win
}

In [156]:
summary_scores_solo = {
    pseudonym: {
        'rt_by_traj_efficiency_first_solo': summary_scores_first_solo[pseudonym],
        'rt_by_traj_efficiency_second_solo': summary_scores_second_solo[pseudonym],
        'experiment_date': solo_dates.get(pseudonym,np.nan)
    }
    for pseudonym in summary_scores_first_solo
}

In [157]:
summary_scores_social

{'FA12': {'rt_by_traj_efficiency_wins': np.float64(0.8115884777483575),
  'rt_by_traj_efficiency_losses': np.float64(0.46916816703501374),
  'experiment_date': '2024-11-12',
  'opponent': 'SL12'},
 'SL12': {'rt_by_traj_efficiency_wins': np.float64(0.5783619884579273),
  'rt_by_traj_efficiency_losses': np.float64(0.5866940803668838),
  'experiment_date': '2024-11-12',
  'opponent': 'FA12'},
 'TE10': {'rt_by_traj_efficiency_wins': np.float64(0.661560012207635),
  'rt_by_traj_efficiency_losses': np.float64(0.5016414178582435),
  'experiment_date': '2024-12-10',
  'opponent': 'TK10'},
 'TK10': {'rt_by_traj_efficiency_wins': np.float64(0.7106118816099372),
  'rt_by_traj_efficiency_losses': np.float64(0.5505248519311575),
  'experiment_date': '2024-12-10',
  'opponent': 'TE10'},
 'AL01': {'rt_by_traj_efficiency_wins': np.float64(0.512412671484463),
  'rt_by_traj_efficiency_losses': np.float64(0.5337082111418647),
  'experiment_date': '2025-04-01',
  'opponent': 'NL01'},
 'NL01': {'rt_by_traj

In [158]:
summary_scores_solo

{'FA12': {'rt_by_traj_efficiency_first_solo': array([0.61555857]),
  'rt_by_traj_efficiency_second_solo': array([0.81822578]),
  'experiment_date': '2024-11-12'},
 'SL12': {'rt_by_traj_efficiency_first_solo': array([0.50449422]),
  'rt_by_traj_efficiency_second_solo': array([0.8163231]),
  'experiment_date': '2024-11-12'},
 'TE10': {'rt_by_traj_efficiency_first_solo': array([0.82483204]),
  'rt_by_traj_efficiency_second_solo': array([0.67217552]),
  'experiment_date': '2024-12-10'},
 'TK10': {'rt_by_traj_efficiency_first_solo': array([0.69867411]),
  'rt_by_traj_efficiency_second_solo': array([0.73031477]),
  'experiment_date': '2024-12-10'},
 'AL01': {'rt_by_traj_efficiency_first_solo': array([0.61013686]),
  'rt_by_traj_efficiency_second_solo': array([0.50137522]),
  'experiment_date': '2025-04-01'},
 'NL01': {'rt_by_traj_efficiency_first_solo': array([0.78322696]),
  'rt_by_traj_efficiency_second_solo': array([0.65765251]),
  'experiment_date': '2025-04-01'},
 'HL19': {'rt_by_traj_e